In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
# Results
#max_depth default is 1 is too low, but a too high value will occur overfitting
max_depthChoice=5
# learning too low makes the process very slow
learning_rateChoice=0.5
n_estimatorsChoice=70

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

In [ ]:
%%script false

from sklearn.model_selection import KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np

predictors = fd.getPredictors(dfTrx)
x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)
train_f1s=[]
test_f1s=[]
range= []

for rate in np.arange(0.1,1.1,0.1):
    print('rate',rate)
    # Instantiate model and k-fold
    estimator = DecisionTreeClassifier(max_depth=max_depthChoice)
    modelClf = AdaBoostClassifier(random_state=42,estimator=estimator, n_estimators=n_estimatorsChoice, learning_rate=learning_rateChoice)
    #modelClf.fit(x_train, y_train)
    modelClf.fit(x_train, y_train,sample_weight=[1 if i == 1 else rate for i in y_train])
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)

    test_preds = modelClf.predict(x_test)
    test_f1 = f1_score(y_test,test_preds)
    #fd.show_confusion_matrix(y_train, train_preds)

    print("train_f1", train_f1)
    print("test_f1 ", test_f1)
    print("dif     ", train_f1-test_f1)
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(rate)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", test_f1s,"test")


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np

rate =0.5
print('rate',rate)
# Instantiate model and k-fold
estimator = DecisionTreeClassifier(max_depth=max_depthChoice)
modelClf = AdaBoostClassifier(random_state=42,estimator=estimator, n_estimators=n_estimatorsChoice, learning_rate=learning_rateChoice)

predictors = fd.getPredictors(dfTrx)
x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)


#modelClf.fit(x_train, y_train)
modelClf.fit(x_train, y_train,sample_weight=[1 if i == 1 else rate for i in y_train])
train_preds = modelClf.predict(x_train)
train_f1 = f1_score(y_train,train_preds)

test_preds = modelClf.predict(x_test)
full_preds = modelClf.predict(dfTrx[predictors])
test_f1 = f1_score(y_test,test_preds)
#fd.show_confusion_matrix(y_train, train_preds)


print(train_f1)
print(test_f1)

fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_train, train_preds)
fd.show_confusion_matrix(y_test, test_preds)
fd.show_confusion_matrix(dfTrx['Class'], full_preds)

fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD07C-AdaBoostClassifierProbaHistogram' if saveImage else None)

dfTrx2 = pd.read_csv('../data/cleaned/WLexport20241118.csv')
full_preds3 = modelClf.predict(dfTrx2[predictors])
fd.show_confusion_matrix(dfTrx2['Class'], full_preds3)


In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result('07-sklearn.ensemble.b-boosting','AdaBoostClassifier','3-sample-weight', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])
